In [22]:
import math
import itertools
import numpy as np
import random
from dataclasses import dataclass
import matplotlib.pyplot as plt
from collections import Counter
import sympy as sp
from sympy import gcd, gcdex
from itertools import product

In [23]:
a_vec = np.zeros(12, dtype=int)
a_vec

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [24]:
G = np.zeros((36, 12), dtype=int)
for i in range(6):
    for j in range(6):
        G[6*i+j, i] = 1 - a_vec[6+i]
        G[6*i+j, 6+j] = a_vec[j]-1

In [25]:
G_a = G[[i for i in range(36) if i not in [3, 8]]]

In [26]:
G_b = G[[3, 8]]

In [27]:
def print_g_matrix(matrix, split_idx=6):
    for row in matrix:
        left = row[:split_idx]
        right = row[split_idx:]
        left_str = " ".join(f"{val:3}" for val in left)
        right_str = " ".join(f"{val:3}" for val in right)
        print(f"{left_str} | {right_str}")

def print_g_smart(matrix, split_idx=6):
    for row in matrix:
        elements = []
        for i, val in enumerate(row):
            if i == split_idx:
                elements.append("|")
            if val == 0:
                elements.append(f"   ·")
            else:
                elements.append(f"{val:4}")
        
        # 行番号を付けて表示
        print(f"{' '.join(elements)}")

In [28]:
ncol = 12
nrow = 6
L=12

長さ4のサイクル

In [29]:
c_4 = []
for r1 in range(nrow):
    for r2 in range(r1 + 1, nrow): # r1より大きいインデックスのみ選ぶ
        for c1 in range(ncol):
            for c2 in range(c1 + 1, ncol): # c1より大きいインデックスのみ選ぶ
                pos1 = [r1, c1]
                pos2 = [r2, c2]
                c_4.append([pos1, pos2])
print(len(c_4))

990


In [30]:
c_6 = []
for r_set in itertools.combinations(list(range(nrow)), 3):
    r1, r2, r3 = r_set # 常に r1 < r2 < r3 となる
    
    # 2. 列を3つ選ぶ (組み合わせ: 12C3 = 220通り)
    for c_set in itertools.combinations(list(range(ncol)), 3):
        # 3. 3つの列の「並び順」をすべて試す (順列: 3! = 6通り)
        # これにより、固定された3行3列の中で可能な6つのサイクルをすべて生成する
        for c_perm in itertools.permutations(c_set):
            c1, c2, c3 = c_perm
            
            # 画像の定義に基づき、3つの位置をリスト化
            # [r1, c1], [r2, c2], [r3, c3]
            cycle_positions = [(r1, c1), (r2, c2), (r3, c3)]
            c_6.append(cycle_positions)
len(c_6)

26400

In [31]:
def get_valid_sequences(elements, length=4):
    
    valid = []
    for seq in product(elements, repeat=length):
        # 隣接チェック (r1!=r2, r2!=r3, r3!=r4, r4!=r1)
        if all(seq[i] != seq[(i+1)%length] for i in range(length)):
            valid.append(seq)
    return valid

def canonicalize(r_seq, c_seq):
    """サイクルを標準化して重複判定を可能にする"""
    # 4つの位置のペアを作成
    positions = tuple((r_seq[i], c_seq[i]) for i in range(4))
    
    # 8つの対称パターン（回転4種、反転4種）を生成
    symmetries = []
    curr = list(positions)
    curr_equiv = get_equiv_pos(curr)
    for _ in range(4):
        curr = curr[1:] + curr[:1]
        symmetries.append(tuple(curr))
        curr_equiv = curr_equiv[1:] + curr_equiv[:1]
        symmetries.append(tuple(curr_equiv))
    # 8つの中で辞書順最小のものを代表とする
    return min(symmetries)

def get_equiv_pos(pos):
    pos1 = tuple((pos[0][0], pos[1][1]))
    pos2 = tuple((pos[3][0], pos[0][1]))
    pos3 = tuple((pos[2][0], pos[3][1]))
    pos4 = tuple((pos[1][0], pos[2][1]))
    return [pos1, pos2, pos3, pos4]

row_seqs = get_valid_sequences(range(nrow))  # 630通り
col_seqs = get_valid_sequences(range(ncol))  # 14652通り
c_8 = []
c_8 = set()
for r in row_seqs:
    for c in col_seqs:
        c_8.add(canonicalize(r, c))
c_8 = list(c_8)

In [32]:
def get_cycle(pos):
    length = len(pos)
    cycle = []
    for i in range(length):
        cycle.append([pos[i][0], pos[i][1]])
        cycle.append([pos[i][0], pos[(i+1)%length][1]])
    return cycle
